In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

from utils import normalize


In [11]:
path = 'population.csv'

df = pd.read_csv(path)
df.head()

C:\Users\A1BY3ZZ\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,NIVGEO,CODGEO,LIBGEO,MOCO,AGEQ80_17,SEXE,NB
0,COM,1001,L'Abergement-Clémenciat,11,0,1,15
1,COM,1001,L'Abergement-Clémenciat,11,0,2,15
2,COM,1001,L'Abergement-Clémenciat,11,5,1,20
3,COM,1001,L'Abergement-Clémenciat,11,5,2,20
4,COM,1001,L'Abergement-Clémenciat,11,10,1,20


In [12]:
df.columns = [normalize(c) for c in df.columns]

In [13]:
df = df[[c for c in df.columns if c != 'nivgeo']]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8536584 entries, 0 to 8536583
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   codgeo     object
 1   libgeo     object
 2   moco       int64 
 3   ageq80_17  int64 
 4   sexe       int64 
 5   nb         int64 
dtypes: int64(4), object(2)
memory usage: 390.8+ MB


In [15]:
df.isna().sum()

codgeo       0
libgeo       0
moco         0
ageq80_17    0
sexe         0
nb           0
dtype: int64

## Exclusion de la Corse

In [16]:
df = df[df['codgeo'].astype(str).apply(lambda x: ('A' not in x) and ('B' not in x))]

In [17]:
df['codgeo'] = df['codgeo'].astype(int)

In [18]:
df['moco'].unique()

array([11, 12, 21, 22, 23, 31, 32], dtype=int64)

## Remplacement des DOM TOM

In [19]:
df = df[df['codgeo'] < 96000]

## Remplacement moco

In [20]:
correspondance_moco = {
    11: 'Enfant d’un couple',
    12: 'Enfant d’une famille monoparentale',
    21: 'Adulte d’un couple sans enfant',
    22: 'Adulte d’un couple avec enfant(s)',
    23: 'Adulte d’une famille monoparentale',
    31: 'Personne vivant hors famille dans un ménage de plusieurs personnes',
    32: 'Personne vivant seule',
    40: 'Personne vivant hors ménage'
    }


for key in correspondance_moco:
    correspondance_moco[key] = normalize(correspondance_moco[key])
    
correspondance_moco

{11: 'enfant_dun_couple',
 12: 'enfant_dune_famille_monoparentale',
 21: 'adulte_dun_couple_sans_enfant',
 22: 'adulte_dun_couple_avec_enfant(s)',
 23: 'adulte_dune_famille_monoparentale',
 31: 'personne_vivant_hors_famille_dans_un_menage_de_plusieurs_personnes',
 32: 'personne_vivant_seule',
 40: 'personne_vivant_hors_menage'}

In [21]:
df['moco'] = df['moco'].replace(correspondance_moco)

In [22]:
df.head()

,codgeo,libgeo,moco,ageq80_17,sexe,nb
0,1001,L'Abergement-Clémenciat,enfant_dun_couple,0,1,15
1,1001,L'Abergement-Clémenciat,enfant_dun_couple,0,2,15
2,1001,L'Abergement-Clémenciat,enfant_dun_couple,5,1,20
3,1001,L'Abergement-Clémenciat,enfant_dun_couple,5,2,20
4,1001,L'Abergement-Clémenciat,enfant_dun_couple,10,1,20


## Remplacement sexe

In [23]:
df['sexe'] = df['sexe'].replace({1: 'homme', 2: 'femme'})
df.head()

,codgeo,libgeo,moco,ageq80_17,sexe,nb
0,1001,L'Abergement-Clémenciat,enfant_dun_couple,0,homme,15
1,1001,L'Abergement-Clémenciat,enfant_dun_couple,0,femme,15
2,1001,L'Abergement-Clémenciat,enfant_dun_couple,5,homme,20
3,1001,L'Abergement-Clémenciat,enfant_dun_couple,5,femme,20
4,1001,L'Abergement-Clémenciat,enfant_dun_couple,10,homme,20


## Remplacement age

In [24]:
ages = df['ageq80_17'].unique()
ages

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
      dtype=int64)

In [25]:
correspondance_age = {
    i: '{}-{}'.format(i, j) for i, j in zip(ages[:-1], ages[1:])
}

correspondance_age[80] = '80+'

correspondance_age

{0: '0-5',
 5: '5-10',
 10: '10-15',
 15: '15-20',
 20: '20-25',
 25: '25-30',
 30: '30-35',
 35: '35-40',
 40: '40-45',
 45: '45-50',
 50: '50-55',
 55: '55-60',
 60: '60-65',
 65: '65-70',
 70: '70-75',
 75: '75-80',
 80: '80+'}

In [26]:
df['ageq80_17'] = df['ageq80_17'].replace(correspondance_age)

In [27]:
df = df.rename(columns={'ageq80_17': 'age', 'moco': 'mode_cohabitation'})

In [28]:
df.head()

,codgeo,libgeo,mode_cohabitation,age,sexe,nb
0,1001,L'Abergement-Clémenciat,enfant_dun_couple,0-5,homme,15
1,1001,L'Abergement-Clémenciat,enfant_dun_couple,0-5,femme,15
2,1001,L'Abergement-Clémenciat,enfant_dun_couple,5-10,homme,20
3,1001,L'Abergement-Clémenciat,enfant_dun_couple,5-10,femme,20
4,1001,L'Abergement-Clémenciat,enfant_dun_couple,10-15,homme,20


## Gestion des villes avec arrondissements

In [29]:
number = re.compile('[0-9]+')
df_arrondissements = df[df['libgeo'].astype(str).apply(lambda x: len(number.findall(x))> 0)]

In [30]:
print(df_arrondissements.shape)
df_arrondissements

(0, 6)


,codgeo,libgeo,mode_cohabitation,age,sexe,nb


In [31]:
df[df['libgeo'] == 'Paris']['codgeo'].unique()

array([75056])

In [32]:
df[df['libgeo'] == 'Marseille']['codgeo'].unique()

array([13055])

In [33]:
df[df['libgeo'] == 'Lyon']['codgeo'].unique()

array([69123])

In [34]:
# on remplace le libgeo de Paris, Lyon et Marseille

df.loc[df['libgeo'] == 'Paris', 'codgeo'] = 75000
df.loc[df['libgeo'] == 'Marseille', 'codgeo'] = 13000
df.loc[df['libgeo'] == 'Lyon', 'codgeo'] = 69000


## Formattage des données


In [36]:
dfs = []

for col in ['age', 'sexe', 'mode_cohabitation']:
    for val in df[col].unique():
        #créer un df pour chque sum de NB par variable
        df_temp = df[df[col] == val].groupby(['libgeo', 'codgeo']).agg({'nb': sum}).reset_index()
        # pour renommer les colonnes 
        df_temp = df_temp.rename(columns={'nb': 'pop_{}_{}'.format(col, val)})
        
        dfs.append(df_temp)

In [37]:
df_total = df.groupby(['libgeo', 'codgeo']).agg({'nb': sum}).reset_index()

df_total = df_total.rename(columns={'nb': 'pop_total'})
df_total.head()

,libgeo,codgeo,pop_total
0,Aast,64001,144
1,Abainville,55001,272
2,Abancourt,59001,436
3,Abancourt,60001,592
4,Abaucourt,54001,290


In [38]:
# pour merger tous les df_temp inclus dans dfs: 
for df_temp in dfs:
    df_total = pd.merge(
        left=df_total,
        right=df_temp,
        on=['libgeo', 'codgeo'],
        how='left'
    )

In [39]:
df_total.head()

,libgeo,codgeo,pop_total,pop_age_0-5,pop_age_5-10,pop_age_10-15,pop_age_15-20,pop_age_20-25,pop_age_25-30,pop_age_30-35,pop_age_35-40,pop_age_40-45,pop_age_45-50,pop_age_50-55,pop_age_55-60,pop_age_60-65,pop_age_65-70,pop_age_70-75,pop_age_75-80,pop_age_80+,pop_sexe_homme,pop_sexe_femme,pop_mode_cohabitation_enfant_dun_couple,pop_mode_cohabitation_enfant_dune_famille_monoparentale,pop_mode_cohabitation_adulte_dun_couple_sans_enfant,pop_mode_cohabitation_adulte_dun_couple_avec_enfant(s),pop_mode_cohabitation_adulte_dune_famille_monoparentale,pop_mode_cohabitation_personne_vivant_hors_famille_dans_un_menage_de_plusieurs_personnes,pop_mode_cohabitation_personne_vivant_seule
0,Aast,64001,144,15,10,10,0,0,0,10,14,16,17,6,18,6,6,6,10,0,74,70,41,0,27,53,0,7,16
1,Abainville,55001,272,4,18,23,22,13,8,8,18,8,13,22,26,55,14,4,4,12,139,133,80,0,80,88,0,0,24
2,Abancourt,59001,436,24,24,32,28,28,28,8,28,40,44,28,44,4,8,16,28,24,216,220,120,20,96,136,16,16,32
3,Abancourt,60001,592,68,53,27,13,40,66,91,8,21,22,49,16,39,27,17,22,13,297,295,176,12,160,179,12,4,49
4,Abaucourt,54001,290,15,25,15,20,0,15,30,25,15,20,20,0,30,20,5,10,25,140,150,50,35,80,60,25,0,40


In [40]:
df_total.shape

(35396, 29)

In [41]:
df_total.isna().sum()

libgeo                                                                                      0
codgeo                                                                                      0
pop_total                                                                                   0
pop_age_0-5                                                                                 0
pop_age_5-10                                                                                0
pop_age_10-15                                                                               0
pop_age_15-20                                                                               0
pop_age_20-25                                                                               0
pop_age_25-30                                                                               0
pop_age_30-35                                                                               0
pop_age_35-40                                               

In [42]:
df_total.to_csv('clean_data/population_clean.csv', index=False)